<a href="https://colab.research.google.com/github/SmartCodar/boto3samples/blob/main/Llama2LlamaIndex_TCS_AR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pypdf
!pip install python-dotenv

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install llama-index


In [5]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM

In [6]:
documents = SimpleDirectoryReader("/content/").load_data()

In [7]:

from llama_index.prompts.prompts import SimpleInputPrompt


system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."



# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [ ]:
!huggingface-cli login

In [ ]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)


In [ ]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

In [12]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [15]:
query_engine = index.as_query_engine()
response = query_engine.query("What are the Property, plant and equipment expences in Standalone Balancesheet")

In [16]:
print(response)

Thank you for asking! Based on the context information provided, the Property, plant and equipment expenses in the Standalone Balance Sheet can be found in Note 10 of the annual report. According to Note 10, the total amount of Property, plant and equipment expenses for the year ended March 31, 2022 was `9,217 crore, while the total amount for the year ended March 31, 2021 was `9,172 crore. I hope this information helps! If you have any other questions, please feel free to ask.


In [ ]:
while True:
  query=input()
  response = query_engine.query(query)
  print(response)

Totally irrelevant question
What is the average age of employees at Jaguar Land Rover?
What is the cost of an iphone
The cost of an iPhone varies depending on the model and storage capacity. According to the annual report of Tata Motors for the financial year 2021-22, the cost of an iPhone is not disclosed. The report only provides information on the company's expenses, including freight, transportation, port charges, and other expenses, but does not provide information on the cost of individual products like iPhones. Therefore, I cannot provide an answer to your question.
